This documents describes the setting up of the database for MDFP experiments performed by Carl Schiebroek. This document shows how the lwreg databse cs_mdfps as well as the postgreSQL schema cs_mdfps_schema_copy is setup. 

In [1]:
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem.Draw import IPythonConsole
from rdkit.Chem import rdDepictor
rdDepictor.SetPreferCoordGen(True)
import os
import json
import uuid

import rdkit
print(rdkit.__version__)
%load_ext sql
import lwreg
from lwreg import standardization_lib
from lwreg import utils

2023.03.3


In [2]:
config = lwreg.utils.defaultConfig()
# set the name of the database we'll work with:
config['dbtype'] = 'postgresql'
config['dbname'] = 'cs_mdfps'
config['host'] = 'lebanon'
config['user'] = 'cschiebroek'
config['password'] = '' # password is saved in our .pgpass
# we don't want to standardize the molecules:
config['standardization'] = standardization_lib.RemoveHs()
# we want to store conformers
config['registerConformers'] = True

In [ ]:
cn = utils._connect(config)
cur = cn.cursor()
#create the schema itself
cur.execute('create schema if not exists cs_mdfps_schema_copy')
#data for the generated conformers
cur.execute('create table cs_mdfps_schema_copy.confid_data(conf_id int primary key, confgen_uuid text)')
cur.execute('create table cs_mdfps_schema_copy.confid_metadata(confgen_uuid text primary key, metadata jsonb)')
#data for the experimental data
cur.execute('create table cs_mdfps_schema_copy.experimental_data(molregno int primary key,logP int, logP_metadata jsonb, VP double precision, VP_metadata jsonb)')
#data for the MD experiment
cur.execute('create table cs_mdfps_schema_copy.md_experiments_metadata(Md_Experiment_uuid text primary key,ff_name text, ff_version text, simulation_type text, MD_engine text, version text, time double precision,git_repo_name text, git_commit_hash text,analysis jsonb,starting_date date)')
#data for the MDFP experiment
cur.execute('create table cs_mdfps_schema_copy.mdfp_experiment_metadata(mdfp_experiment_uuid text primary key,md_experiment_uuid text,parameters jsonb)')
cur.execute('create table cs_mdfps_schema_copy.mdfp_experiment_data(conf_id int, mdfp_experiment_uuid text,MDFP jsonb,mdfp_conf_uuid text primary key,md_experiment_uuid text)')
# cn.commit()